# Demonstration of EPISTEM-x Module 
This notebook contain the implementation of the source code for each module in the EPISTEM land cover mapping framework

## Library import and earth engine initialization
If you have earth engine account you could used that to authenticate and initialize the earth engine. However, if you did not have the account, service account initialization is avaliable

In [ ]:
#This code is used if the notebook is implemented in github codespace. Just remove the (#)
#!python -m pip install ../epistemx --quiet

In [32]:
import ee 
import epistemx

#Option 1: Manual authenticate using personal account
#Instructions for manual authentication
epistemx.print_auth_instructions()
#uncomment the below line and follow earth engine authentication process
#epistemx.authenticate_manually()

#Option 2: Autheticate using service account (json file)
service_account_path = '../auth/ee-rg2icraf-ecab9c534f91.json'
success = epistemx.initialize_with_service_account(service_account_path)

if success:
    print("Earth Engine initialized with service account successfully!")
else:
    print("Service account initialization failed. Try to authenticate earth engine manually")

#Check authentication status
status = epistemx.get_auth_status()
print(f"Initialized: {status['initialized']}")
print(f"Authenticated: {status['authenticated']}")
if status['project']:
    print(f"Project: {status['project']}")


    EARTH ENGINE AUTHENTICATION NOTES:
    
    1. Make sure you already have a google cloud project that has enable the Earth Engine API and registered to 
       commercial or non-commercial use. For more information visit: https://developers.google.com/earth-engine/guides/access 
    
    2. you can authenticate programmatically by calling: from epistemx.ee_config import authenticate_manually
       authenticate_manually()
    
    3. This will open a web browser. Sign in with your Google account that has Earth Engine access.
    
    4. Copy the authorization code from the browser and paste it in the terminal.
    
    
    For more details, visit: https://developers.google.com/earth-engine/guides/python_install
    


2025-10-27 21:48:32,412 - epistemx.ee_config - INFO - Earth Engine initialized with service account for project: ee-rg2icraf


Earth Engine initialized with service account successfully!
Initialized: True
Authenticated: True
Project: projects/ee-rg2icraf/assets/Admin_NTT


In [2]:
#!python -m pip install ../epistemx --quiet
import geemap
from epistemx.module_1 import Reflectance_Data, Reflectance_Stats
from epistemx.helpers import get_aoi_from_gaul

## Module 1: Acquisition of Near-Cloud-Free Satellite Imagery

### System Response 1.1: Area of Interest Definition

In [3]:
#Set the country and province for the AOI using GAUL admin boundaries
aoi = get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan")
#Alternatively, used geemap_shp_to_ee to directly used shapefile in your local machine

### System Response 1.2: Search and Filter Imagery
The EPISTEM source code supports Landsat mission data, ranging from Landsat 1 to Landsat 9. For Landsat 1 - 3, the avaliable data is corrected radiance reflectance. The Landsat 5-9 used here is collection 2 surface reflectance (SR) analysis ready data.

The retrival logic used here is as follow:
1. Retrive multispectral bands (band 1 - 7) from landsat collection 2 SR data (if avaliable)
2. Retrive thermal band from landsat collection 2 TOA data 
3. Create temporal composite for each data 
4. Stacked the final two data into a earth engine image (ee.image)

In [4]:
#========== FIRST RETRIVE THE MULTISPECTRAL BAND===========
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2017-12-31'
#get the image collection and corresponding statistics
landsat_data, meta = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=False)
#create mosaic between image collection, and clip based on AOI
mosaic_landsat = landsat_data.mosaic().clip(aoi)
#Alternatively you can use temporal aggregation (ee reducer) to create mode cloudless imagery
median_landsat = landsat_data.median().clip(aoi)
#visualization parameter
l8_sr_visparam = {'min': 0,'max': 0.4,'gamma': [0.95, 1.1, 1],'bands':['NIR', 'RED', 'GREEN']}
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map.addLayer(median_landsat, l8_sr_visparam, 'L8 SR Median')
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')
# set center of the map in the area of interest
Map.centerObject(aoi, 7)

2025-10-27 21:25:42,710 - Reflectance_Data - INFO - ReflectanceData initialized.
2025-10-27 21:25:42,712 - Reflectance_Data - INFO - Starting data fetch for Landsat 8 Operational Land Imager Surface Reflectance
2025-10-27 21:25:42,712 - Reflectance_Data - INFO - Date range: 2017-01-01 to 2017-12-31
2025-10-27 21:25:42,713 - Reflectance_Data - INFO - Cloud cover threshold: 40%
2025-10-27 21:25:42,713 - Reflectance_Data - INFO - detailed statistics will not be computed
2025-10-27 21:25:42,714 - Reflectance_Stats - INFO - Reflectance Stats initialized.
2025-10-27 21:25:42,715 - Reflectance_Data - INFO - Filtered collection created (use compute_detailed_stats=True for more information)


In [5]:
#retive thermal bands from TOA
thermal_bands, thermal_stats = optical_reflectance.get_thermal_bands(aoi, start, end, cloud_cover=40, compute_detailed_stats=False)
median_thermal = thermal_bands.median().clip(aoi)
thermal_vis = {
    'min': 286,
    'max': 300,
    'gammma': 0.4
}
#stacked all landsat bands
stacked_landsat = median_landsat.addBands(median_thermal)
#visualize the thermal bands and multispectral bands
Map.addLayer(median_thermal, thermal_vis, "Thermal Bands")
Map

2025-10-27 21:25:47,461 - Reflectance_Stats - INFO - Reflectance Stats initialized.
2025-10-27 21:25:47,462 - Reflectance_Data - INFO - Starting thermal data fetch for Landsat 8 Top-of-atmosphere reflectance
2025-10-27 21:25:47,463 - Reflectance_Data - INFO - Date range: 2017-01-01 to 2017-12-31
2025-10-27 21:25:47,463 - Reflectance_Data - INFO - Cloud cover threshold: 40%
2025-10-27 21:25:47,464 - Reflectance_Data - INFO - Fast mode enabled - detailed statistics will not be computed
2025-10-27 21:25:47,465 - Reflectance_Data - INFO - Filtered collection created (use compute_detailed_stats=True for detailed info)


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### Image retrival report (optional)

In [6]:
#intialize the statistic class
stats = Reflectance_Stats()
#get the retrival report and automatically print them
retrival_report = stats.get_collection_statistics(landsat_data, print_report=True)

2025-10-27 21:25:49,757 - Reflectance_Stats - INFO - Reflectance Stats initialized.


           Landsat Data Collection Retrival Report
Total Images Found: 53
Date Range: 2017-01-13 to 2017-12-17
Unique WRS Tiles: 10

Scene Cloud Cover Statistics:
------------------------------
Average Cloud Cover: 26.8%
Minimum Cloud Cover: 2.9%
Maximum Cloud Cover: 40.0%

WRS Path/Row Tiles:
------------------------------
Path 123/Row 062
Path 123/Row 063
Path 124/Row 061
Path 124/Row 062
Path 124/Row 063
Path 124/Row 064
Path 125/Row 061
Path 125/Row 062
Path 125/Row 063
Path 126/Row 062

Available Acqusition Date:
------------------------------
Date range: 2017-01-13 to 2017-12-17
(53 total acquisition dates)

Scene IDs (first 10):
------------------------------
• LC08_123062_20170405
• LC08_123062_20170421
• LC08_123062_20170726
• LC08_123062_20170912
• LC08_123062_20171014
• LC08_123062_20171217
• LC08_123063_20170710
• LC08_123063_20170827
• LC08_123063_20170912
• LC08_123063_20171030
... and 43 more scenes



### System Response 1.3: Imagery Download

In [ ]:
export_task = ee.batch.Export.image.toDrive(
    image=stacked_landsat,
    description='Landsat_Median_composite_2017_Sumsel',
    folder='Earth Engine',
    fileNamePrefix='Landsat_Median_composite_2017_Sumsel',
    scale=30,
    region=aoi,  # or aoi.geometry()
    maxPixels=1e13
)
export_task.start()
import time

while export_task.active():
    print('Exporting... (status: {})'.format(export_task.status()['state']))
    time.sleep(10)

print('Export complete (status: {})'.format(export_task.status()['state']))

## Module 2:  Land-cover classification Scheme
Three approach are provided to handle classification scheme:
1. Upload a csv file 
2. Manual input the classification scheme
3. Use default classification scheme (RESTORE+ project)

### Import the module

In [6]:
from epistemx.module_2 import LULC_Scheme_Manager
#Initialize the LULC Scheme Manager
manager = LULC_Scheme_Manager()
print("Land Cover Classification Scheme Manager initialized!")
print(f"Current class count: {manager.get_class_count()}")
#Temporary function to display the classiifcation scheme in notebook
#Display current classification scheme
def display_classification_scheme(manager):
    """Display the current classification scheme in a readable format"""
    if not manager.has_classes():
        print("No classes defined yet.")
        return
    
    print("\n=== Current Classification Scheme ===")
    df = manager.get_dataframe()
    print(df.to_string(index=False))
    
    return df

# Display the scheme
df = display_classification_scheme(manager)

Land Cover Classification Scheme Manager initialized!
Current class count: 0
No classes defined yet.


### System Response 2.1a: Upload Classification Scheme

In [ ]:
import pandas as pd
#Reset manager for CSV upload example
manager = LULC_Scheme_Manager()
#path to csv 
csv_path = "../test_data/Example_Classification_scheme.csv"

print("=== CSV Upload Process ===")

# Load the CSV
df = pd.read_csv(csv_path, sep=None, engine="python")
print("Loaded CSV:")
print(df)

# Auto-detect columns
id_col, name_col, color_col = manager.auto_detect_csv_columns(df)
print(f"\nAuto-detected columns:")
print(f"ID column: {id_col}")
print(f"Name column: {name_col}")
print(f"Color column: {color_col}")

=== CSV Upload Process ===
Loaded CSV:
    ﻿No                       Class_name  Class_id
0     1               Rubber monoculture        15
1     2                       Settlement        16
2     3             Oil palm monoculture        11
3     4                       Water Body        19
4     5                Acacia plantation         1
5     6                       Rice Field        13
6     7                     Mixed Garden        10
7     8                Rubber agroforest        14
8     9   Logged Over Forest Low Density         8
9    10                            Shrub        17
10   11                            Grass         6
11   12         Logged over swamp forest         9
12   13  Logged over forest-high density         7
13   14                Coffee Agroforest         5
14   15              Coconut monoculture         4
15   16                      Other Crops        12
16   17                     Cleared Land         3
17   18                   Tea plantation   

In [8]:
# Process CSV upload
success, message = manager.process_csv_upload(df, id_col, name_col, color_col)
if success:
    print(f"✅ {message}")
    
    # Finalize the upload
    success, message = manager.finalize_csv_upload()
    if success:
        print(f"✅ {message}")
    else:
        print(f"❌ {message}")
else:
    print(f"❌ {message}")

# Display the loaded scheme
display_classification_scheme(manager)

✅ Successfully loaded 19 classes from CSV with auto-generated colors
✅ Classification scheme created with 19 classes

=== Current Classification Scheme ===
 ID                Land Cover Class Color Palette
  1               Acacia plantation       #FFEAA7
  2                            Cane       #AED6F1
  3                    Cleared Land       #F9E79F
  4             Coconut monoculture       #D7BDE2
  5               Coffee Agroforest       #85C1E9
  6                           Grass       #F8C471
  7 Logged over forest-high density       #F1948A
  8  Logged Over Forest Low Density       #BB8FCE
  9        Logged over swamp forest       #82E0AA
 10                    Mixed Garden       #98D8C8
 11            Oil palm monoculture       #45B7D1
 12                     Other Crops       #A3E4D7
 13                      Rice Field       #DDA0DD
 14               Rubber agroforest       #F7DC6F
 15              Rubber monoculture       #FF6B6B
 16                      Settlement       #4

,ID,Land Cover Class,Color Palette
0,1,Acacia plantation,#FFEAA7
1,2,Cane,#AED6F1
2,3,Cleared Land,#F9E79F
3,4,Coconut monoculture,#D7BDE2
4,5,Coffee Agroforest,#85C1E9
5,6,Grass,#F8C471
6,7,Logged over forest-high density,#F1948A
7,8,Logged Over Forest Low Density,#BB8FCE
8,9,Logged over swamp forest,#82E0AA
9,10,Mixed Garden,#98D8C8


### System Response 2.1b: Manual Scheme Definition

In [10]:
#Reset manager for manual input example
manager = LULC_Scheme_Manager()
#Manually add the class
print("=== Manual Class Addition ===")

#Example of class to add
classes_to_add = [
    (1, "Hutan Lahan Kering", "#0E6D0E"),
    (2, "Pertanian Lahan Kering", "#E8F800"),
    (3, "Permukiman", "#F81D00"),
    (4, "Badan Air", "#1512F3"),
    (5, "Pertanian Lahan Basah", "#")
]

for class_id, class_name, color_code in classes_to_add:
    success, message = manager.add_class(class_id, class_name, color_code)
    if success:
        print(f"✅ {message}")
    else:
        print(f"❌ {message}")

print(f"\nTotal classes: {manager.get_class_count()}")

=== Manual Class Addition ===
✅ Class 'Hutan Lahan Kering' (ID: 1) added successfully!
✅ Class 'Pertanian Lahan Kering' (ID: 2) added successfully!
✅ Class 'Permukiman' (ID: 3) added successfully!
✅ Class 'Badan Air' (ID: 4) added successfully!
✅ Class 'Pertanian Lahan Basah' (ID: 5) added successfully!

Total classes: 5


In [11]:
# Example: Edit an existing class
print("=== Editing a Class ===")

# Edit the first class (index 0)
class_to_edit = manager.edit_class(0)
if class_to_edit:
    print(f"Editing class: {class_to_edit}")
    
    # Update the class with new information
    success, message = manager.add_class(1, "HUtan Lahan Rendah", "#004D00")
    if success:
        print(f"✅ {message}")
    else:
        print(f"❌ {message}")

# Display updated scheme
display_classification_scheme(manager)

=== Editing a Class ===
Editing class: {'ID': 1, 'Class Name': 'Hutan Lahan Kering', 'Color Code': '#0E6D0E'}
✅ Class 'HUtan Lahan Rendah' (ID: 1) updated successfully!

=== Current Classification Scheme ===
 ID       Land Cover Class Color Palette
  1     HUtan Lahan Rendah       #004D00
  2 Pertanian Lahan Kering       #E8F800
  3             Permukiman       #F81D00
  4              Badan Air       #1512F3
  5  Pertanian Lahan Basah             #


,ID,Land Cover Class,Color Palette
0,1,HUtan Lahan Rendah,#004D00
1,2,Pertanian Lahan Kering,#E8F800
2,3,Permukiman,#F81D00
3,4,Badan Air,#1512F3
4,5,Pertanian Lahan Basah,#


### System Response 2.1c: Template Classification Scheme

In [12]:
# Reset manager for default scheme example
manager = LULC_Scheme_Manager()

print("=== Available Default Schemes ===")
default_schemes = manager.get_default_schemes()

for scheme_name, classes in default_schemes.items():
    print(f"\n{scheme_name}: {len(classes)} classes")
    for class_data in classes:
        print(f"  - ID {class_data['ID']}: {class_data['Class Name']} ({class_data['Color Code']})")

=== Available Default Schemes ===

RESTORE+ Project: 10 classes
  - ID 1: Natural Forest (#0E6D0E)
  - ID 2: Agroforestry (#F08306)
  - ID 3: Monoculture Plantation (#38E638)
  - ID 4: Grassland or Savanna (#80DD80)
  - ID 5: Shrub (#5F972A)
  - ID 6: Paddy Field (#777907)
  - ID 7: Cropland (Palawija, Horticulture) (#E8F800)
  - ID 8: Settlement (#F81D00)
  - ID 9: Cleared Land (#E9B970)
  - ID 10: Waterbody (#1512F3)


In [13]:
# Load the RESTORE+ default scheme
scheme_name = "RESTORE+ Project"
success, message = manager.load_default_scheme(scheme_name)

if success:
    print(f"✅ {message}")
else:
    print(f"❌ {message}")

# Display the loaded scheme
display_classification_scheme(manager)

✅ Loaded RESTORE+ Project with 10 classes

=== Current Classification Scheme ===
 ID                  Land Cover Class Color Palette
  1                    Natural Forest       #0E6D0E
  2                      Agroforestry       #F08306
  3            Monoculture Plantation       #38E638
  4              Grassland or Savanna       #80DD80
  5                             Shrub       #5F972A
  6                       Paddy Field       #777907
  7 Cropland (Palawija, Horticulture)       #E8F800
  8                        Settlement       #F81D00
  9                      Cleared Land       #E9B970
 10                         Waterbody       #1512F3


,ID,Land Cover Class,Color Palette
0,1,Natural Forest,#0E6D0E
1,2,Agroforestry,#F08306
2,3,Monoculture Plantation,#38E638
3,4,Grassland or Savanna,#80DD80
4,5,Shrub,#5F972A
5,6,Paddy Field,#777907
6,7,"Cropland (Palawija, Horticulture)",#E8F800
7,8,Settlement,#F81D00
8,9,Cleared Land,#E9B970
9,10,Waterbody,#1512F3


### System Response 2.2: Download classification scheme

In [ ]:
print("=== Export Classification Scheme ===")
#Convert the selected  classification scheme manager to dataframe
classification_df = manager.get_dataframe()
print("Classification DataFrame:")
print(classification_df)
#Save the file
output_path = '../Selected_LC_Classification_Scheme.csv'
classification_df.to_csv(output_path, index=False)
print(f"\n✅ Classification scheme saved to: {output_path}")

=== Export Classification Scheme ===
Classification DataFrame:
   ID                   Land Cover Class Color Palette
0   1                     Natural Forest       #0E6D0E
1   2                       Agroforestry       #F08306
2   3             Monoculture Plantation       #38E638
3   4               Grassland or Savanna       #80DD80
4   5                              Shrub       #5F972A
5   6                        Paddy Field       #777907
6   7  Cropland (Palawija, Horticulture)       #E8F800
7   8                         Settlement       #F81D00
8   9                       Cleared Land       #E9B970
9  10                          Waterbody       #1512F3

✅ Classification scheme saved to: ../classification_scheme_output.csv
CSV data size: 306 bytes
